In [ ]:
suppressMessages({
    library(data.table)
    library(dplyr)
    library(tidyverse)
    library(ggplot2)
    library(readxl)
    library(purrr)
    library(regioneR)
    library(BSgenome.Hsapiens.UCSC.hg19)
    library(GenomicRanges)
    library(TxDb.Hsapiens.UCSC.hg38.knownGene)
    library(biomaRt)
})



In [ ]:
annotation_groups <- fread('../../input/human_cfs/_m/human_cfs_gandharva_gff3.tsv')# %>%


In [ ]:
paths <- c('human_cfs')
paths

In [ ]:
annotation_groups %>% 
    dplyr::select(seqnames, start, end) %>%
    makeGRangesFromDataFrame(ignore.strand = T)
#annotation_groups

In [ ]:
custom_hg38 <- fread('../../input/custom_hg38/_m/hg38_blacklist_24chr.bed') %>%
    as.data.frame() %>%
    dplyr::rename(seqnames = V1, start = V2, end = V3) %>%
    makeGRangesFromDataFrame()

In [ ]:
files_path <- Sys.glob('/ceph/projects/tomoyo_SETD1A_cut-tag_fetalbrain/analysis/qc_manuscript_tables/setd1a_bound_otherregions/_m/*.tsv')
files_path

In [ ]:
df_samples <- files_path %>% 
                map_df(~fread(.))

In [ ]:
sample_list <- unique(df_samples$peakNames)
sample_list <- sample_list[order(sample_list)]
sample_list

In [ ]:
for (i in seq_along(sample_list)){
    
    df <- df_samples %>% 
                filter(peakNames == sample_list[i]) %>%
                dplyr::select(seqnames,start,end, strand, peakNames) %>%
                makeGRangesFromDataFrame(keep.extra.columns = T)
    
    samples <- unique(df$peakNames)
    
    
    annotation_granges <- makeGRangesFromDataFrame(annotation_groups,keep.extra.columns = T)

        
    print(samples)    
    print(numOverlaps(df, annotation_granges, count.once=TRUE))
        
    pt1 <- permTest(A=df, B=annotation_granges, ntimes=10000,genome=custom_hg38,randomize.function=randomizeRegions,
               evaluate.function=numOverlaps, count.once=TRUE, mc.cores=8)
        
    sink(paste0(samples,'_permtest.log')); print(pt1$numOverlaps);sink()
 
    pz_zscore <- localZScore(A=df, B=annotation_granges, pt=pt1, window=10000, step=200, count.once=TRUE)
        

    png(paste0(samples,'_permtest.png'),width = 7,height=7,units = 'in',res = 300)
    plot(pt1)
    dev.off()
        
    png(paste0(samples,'_permzscore.png'),width = 7,height=7,units = 'in',res = 300)
    plot(pz_zscore)
    dev.off()
        
        
    pdf(paste0(samples,'_permtest.pdf'),width = 7,height=7)
    plot(pt1)
    dev.off()

    pdf(paste0(samples,'_permzscore.pdf'),width = 7,height=7)
    plot(pz_zscore)
    dev.off()
    
        }



In [ ]:
sessionInfo()